In [2]:
import pandas as pd

In [3]:
# We're off to a pretty bad start. I have no idea where this data comes from. I assume this is data that you somehow created by transforming
# data from the 3 datasets I got from running the .py files in this project, but I really have no idea how this happened or how I could recreate
# it so I'm going to be doing a lot of shooting in the dark from here on out. 

'''
That was my bad I forgot to include you should run the salary_api.py file 
to pull the data it should make the data folder and put in the data. 
'''

data = pd.read_csv("data/SalaryData.csv")

# Cleaning 

In [4]:
# I'll start by saying that I believe your target variable aka 'y' aka the dependent variable aka the metric you would like to predict is 
# discrepancy ratio, based on what I can deduce by reading through this file. Anything I write from here on out is based on this assumption which 
# at this point is just that. From what I've put together at this point my main concern with this as far as manifesting this into an ML project is 
# figuring out what x (x is almost always a collection of things,) aka the predictor variables aka independent variables might be. Let's imagine 
# we're doing multiple regression, we need a formula along the lines y = m1x1 + m2x2 + m3x3 + mnxn + b, with b being the y intercept and the xs 
# being our predictor variables (n in this instance implies an unknown number of predictors.) So basically our concern at this point is finding 
# variables correlated with y which can be used to predict it.

'''
y (target variable): Discrepancy_Ratio
x (predictor variables): YTD_Total, Salary_Total, CalYear

I think is the idea here? 

'''

cols_to_drop = ['Employee_Name', 'jobTitle', 'Regular_Rate', 'Overtime_Rate', 'Other', 'ObjectId']
data = data.drop(columns=cols_to_drop)
data['Salary_Total'] = data['Annual_Rate'] + data['Incentive_Allowance']

# I'm interested in this incentive allowance metric and where it came from. It might bear fruit using it as a predictor if we can better
# understand it's role and correlation to discrepancy/discrepancy ratio.
# I think both of these drops are misguided in the quest for predictor variables frankly. Though we have above created a metric used to
# represent them we are watering down their value as predictors by creating the above aggregate.
'''
I added it to the salary total as thats the total spend for the employee. 
The Incentive_Allowance is for uniforms ect. I assumed it would be best to
add them together to get the full expected annual cost of the employee. 
'''

final_drop = ['Annual_Rate', 'Incentive_Allowance']
data = data.drop(columns=final_drop)

# I will note when using .fillna() it is always expected that you would have some rationale for your strategy. I think it's probably pretty
# straightforward here, but do anticipate this being questioned. Many would likely .fillna() on a column by column basis filling in the median
# value relative to the column (presuming 'continuous' or numeric values which could hypothetically range from 0 to infinity.) I'll also note
# there is a special kind of regression known as 'poisson' regression which might be worth considering for your purposes...It is particularly 
# useful when the target variable is known to not be truly continuous (ala -infinity to infinity) but known to fall within a range 
# characterized by 0 to 1 as this is how it works under the hood though in practice this can be used for any range. Poisson is particuarly 
# powerful when working with probabilities.
# https://en.wikipedia.org/wiki/Poisson_regression

'''
I would suspect the 0 here would be someone left while still being an staff on paper 
I was going on the assumption it would be best to have a 0 but maybe fill will expected salary?
'''

data = data.fillna(0)
data['Department'] = data['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')

In [5]:
# This would be an insanely small dataset to do any kind of 
# ML with both in rows and columns but most concerningly in 
# columns. It makes a nice aggregate for example for a PowerPoint 
# slide, but would not be ideal to feed to a model.

'''
Yeah I think doing this as an individual department with all the 
employees would be better. 

From above is this a good target? 
y (target variable): Discrepancy_Ratio
x (predictor variables): YTD_Total, Salary_Total, CalYear

'''

data.head()

,CalYear,Department,YTD_Total,Salary_Total
0,2023,OMB Finance,1600.00,5200.00
1,2023,Louisville Free Public Library,30657.64,30960.00
2,2023,Louisville Metro Police Department,41457.53,41457.52
3,2023,Louisville Metro Police Department,19823.87,16912.00
4,2023,Louisville Metro Police Department,58513.88,61796.80


In [6]:
# final_salary = data.groupby('Department')[['YTD_Total', 'Salary_Total']].sum().reset_index()
final_salary = data.groupby(['CalYear', 'Department'])[['YTD_Total', 'Salary_Total']].sum().reset_index()


# Lmpd department view

<font color='red'>This text is red
Would this be the ideal set to work with below? 

From above is this a good target? 
y (target variable): Discrepancy_Ratio or Discrepancy	
x (predictor variables): YTD_Total, Salary_Total, CalYear
</font>

In [15]:
indv = pd.read_csv('data/SalaryData.csv')

In [ ]:
cols_to_drop = ['Regular_Rate', 'Overtime_Rate', 'Other', 'ObjectId']
indv = indv.drop(columns=cols_to_drop)
indv['Salary_Total'] = indv['Annual_Rate'] + indv['Incentive_Allowance']
final_drop = ['Annual_Rate', 'Incentive_Allowance']
indv = indv.drop(columns=final_drop)
indv = indv.fillna(0)
indv['Department'] = indv['Department'].replace('Louisville Metro Police', 'Louisville Metro Police Department')
indv = indv[indv['CalYear'] != 2024].copy()
indv = indv[indv['Department'] == 'Louisville Metro Police Department'].copy()
indv['Discrepancy'] = indv['YTD_Total'] - indv['Salary_Total']
indv['Discrepancy_Ratio'] = indv['Discrepancy'] / indv['Salary_Total']

In [20]:
indv.head(2)

,CalYear,Employee_Name,Department,jobTitle,YTD_Total,Salary_Total,Discrepancy,Discrepancy_Ratio
2,2023,"Bratcher, Elaine",Louisville Metro Police Department,Clerk II-Police,41457.53,41457.52,0.01,2.412108e-07
3,2023,"Jackson, Nila Marie",Louisville Metro Police Department,Traffic Guard I,19823.87,16912.00,2911.87,1.721777e-01
